In [1]:
try:
    import urllib2 as urllib
except ImportError:
    import urllib.request as urllib
import json
import os
import datetime
import pandas as pd
import numpy as np
import shapely.wkt
import geopandas as gpd
from shapely.geometry import Point
import os.path



In [2]:
def cambiar_fecha(fecha):
    return datetime.datetime.fromtimestamp(int(fecha)).strftime('%d-%m-%Y')

def queryInstagramAPI():
    token = os.getenv('CARTOGRAFTOKEN')
    url = 'https://api.instagram.com/v1/tags/cartograf/media/recent?access_token=' + token


    response = urllib.urlopen(url)
    rawData = response.read().decode("utf-8")
    parseData = json.loads(rawData)
    data = parseData['data']
    return data

In [3]:
data = queryInstagramAPI()

In [4]:
usuarios = []
fotos=[]
fechas = []
geometry = []
popUpContent = []
for i in range(len(data)):
    usuarios.append(data[i]['caption']['from']['username']) 
    foto = data[i]['images']
    
    fotos.append(foto)
    fechas.append(cambiar_fecha(data[i]['created_time']))
    
    
    try:    
        punto = Point(data[i]['location']['longitude'],data[i]['location']['latitude'])
        geometry.append(punto)
    except:
        geometry.append(np.nan)
        
        
    grande = foto['standard_resolution']['url']
    chica = foto['low_resolution']['url']
    tamanio = str(foto['low_resolution']['height'])
    
    popUp = '''<b>INTERVENCION CALLEJERA</b> <br> <a href="''' + grande + \
    '''" target="_blank"> <img src="''' + chica + \
    '''" height="''' + tamanio + '''"></a> ''' 
    
    popUpContent.append(popUp)

In [5]:
df = pd.DataFrame({'usuarios':usuarios,
                   #'fotoChica':fotosChicas,
                   #'fotosStandard':fotosStandard,
                   #'fotos':fotos,
                  'fechas':fechas,
                   'popUpContent':popUpContent,
                 'geometry':geometry
                  })
df.dropna(inplace=True)
df

,fechas,geometry,popUpContent,usuarios
0,09-08-2017,POINT (-58.4330381 -34.5879633),"<b>INTERVENCION CALLEJERA</b> <br> <a href=""ht...",moropisa_
1,09-08-2017,POINT (-58.42757 -34.61469),"<b>INTERVENCION CALLEJERA</b> <br> <a href=""ht...",moropisa_


In [6]:
#create GeoDataFrame with linestrings as geometry
crs = {'init': 'epsg:4326'}
gdf = gpd.GeoDataFrame(df, crs=crs, geometry=df.geometry)


In [7]:
filename = '../data/data.geojson'
if os.path.exists(filename):
    os.system('rm '+filename)
gdf.to_file(filename,driver="GeoJSON")